<a href="https://colab.research.google.com/github/HpDiniz/Leitor-de-PDF/blob/main/PDF_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfminer --quiet
!pip install PyPDF2 --quiet

In [24]:
from google.colab import files
file_objects = files.upload() # CRA BRF - MAI/2031 não foi pêgo

Saving Extrato XP - Cliente 01.pdf to Extrato XP - Cliente 01 (5).pdf
Saving Extrato XP - Cliente 02.pdf to Extrato XP - Cliente 02 (3).pdf
Saving Gestao_ExtratoConsolidado Modal - Cliente 04.pdf to Gestao_ExtratoConsolidado Modal - Cliente 04 (3).pdf


# Gerar consolidado

In [32]:
import io
import re
import os
import cv2
import uuid
import base64
import PyPDF2
import openpyxl
import pandas as pd
from datetime import datetime
from flask import Flask, request
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

In [33]:
def get_random_filename(extension):

    if not os.path.isdir("Temp"):
        os.makedirs("Temp")

    return "Temp/" + str(uuid.uuid4()) + '.' + extension

def get_response():

    response = []
    for file in file_objects.keys():

        with open(file, "rb") as f:
            pdf_content = f.read()

        base64_pdf_content = base64.b64encode(pdf_content)

        response.append({
            'FileName': file,
            'Institution': "xxx",
            'Value': base64_pdf_content
        })

    return response

# Converte o conteúdo do PDF para texto
def get_pdfminer_text(path):

    retstr = io.StringIO()
    laparams = LAParams()
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)

    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    for page in PDFPage.get_pages(fp, set(), maxpages=0, password="",caching=True, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()

    return text.strip()

def get_pypdf_text(path):

    text, reader = "", PyPDF2.PdfReader(path)
    for page in reader.pages:
        text = text + page.extract_text()

    return text.strip()

def check_ativo_name(text_pdfminer, ativo):

    # Remove espaços extras do ativo
    ativo = re.sub('\s+',' ', ativo.strip())

    # Obtém todas as linhas obtidas por uma quebra de linha dupla
    pdfminer_lines = text_pdfminer.split("\n\n")

    # Remove tabs, quebras de linhas e espaços duplos de cada posição
    for i in range(len(pdfminer_lines)):
        pdfminer_lines[i] = re.sub('\s+',' ', pdfminer_lines[i].strip())

    # Cria uma cópia do nome do ativo
    ativo_left = ativo
    ativo_right = ativo

    #Loop até quando for possível diminuir o nome do ativo
    while(len(ativo_left) > 1):

        # Se o nome do ativo não possuir espaços, não há o que fazer
        if ' ' not in ativo_left and ' ' not in ativo_right:
             return ativo

        # Procura o nome do ativo nas linhas do pdfminer
        if ativo_left in pdfminer_lines:
            return ativo_left

         # Procura o nome do ativo nas linhas do pdfminer
        if ativo_right in pdfminer_lines:
            return ativo_right

        # Remove o último caracter do {ativo_left} e conitnua a busca
        ativo_left = ativo_left[:-1]
        ativo_right = ativo_right[1:]
    
    # Retorna o {ativo}, caso não tenha sido encontrado nada
    return ativo

def get_tipo_extrato(text_pypdf, text_pdfminer):

    if(len(text_pypdf) > 99 and len(text_pdfminer) > 99):

        text_1 = re.sub('\s+','', text_pypdf[0:100].lower())
        text_2 = re.sub('\s+','', text_pdfminer[0:100].lower())

        if 'extratodecotista' in text_1 and 'extratodecotista' in text_2:
            return "Extrato de Cotista XP"
        elif 'posição&performance' in text_1 and 'posição&performance' in text_2:
            return "Posição e Performance XP"
        elif 'extratoconsolidadoinvestimentos' in text_2:
            return "Extrato Consolidado Modal"
        elif 'posiçãoconsolidadada' in text_2:
            return "Posição Consolidada XP"

    return ""

def obtem_classificacao(ativo):

    # Verifica se é uma ativo no formato de Ticker
    founded = re.search(r'\b[A-Z]{4}(3|4|5|6|11|32|33|34|35)\b', ativo, flags=(re.IGNORECASE))
    if founded:
        return "Ações"
    
    # Define as palavras chave para cada Classificação
    dict_classificacoes = {
        "Ações": [
            "FIC FIA","FIA","FI Ações", "FI Açoes"
        ],
        "Renda Fixa": [
            "Renda Fixa","RF","CDB","LC","LF","LFSN","LFSC","LCA","LCI","CRA",
            "CRI","Deb","Debenture","Debênture","Tesouro","Pré","Pós","Pre",
            "Pos","IPCA","IPCA+","LTN","NTN","NTN-B","NTNB","DI","REF DI",
            "CP","Credito Privado","Crédito Privado","LP","Longo Prazo"
        ],
        "Renda Fixa (Previdência)": [
            "Previdencia","Previdência","Prev","VGBL","PGBL","VGBL/PGBL"
        ],
        "Multimercado": [
            "FIC FIM","FIM","MM","Multimercado","Multi","COE","Multiestratégia",
            "Multiestratégia"
        ]
    }

    classes = []
    # Percorre cada uma das classificações
    for classificacao in dict_classificacoes:

        # Percorre as palavras-chave de cada uma das classificações
        for key in dict_classificacoes[classificacao]:

            # Verifica se a palavra-chave atual existe no nome do ativo
            founded = re.search(r'\b' + key + r'\b', ativo, flags=(re.IGNORECASE))
            if founded and not classificacao in classes:
                classes.append(classificacao)

    if len(classes) > 0:
        return ' / '.join(classes)

    # Se nenhuma classificação foi encontrada, verifica se é um ETF de investimento estrangeiro
    founded = re.search(r'\b[A-Z]{3}\b', ativo, flags=(re.IGNORECASE))
    if founded:
        return "Ações"

    return "***"

def convert_base64_pdf_to_text(value):

    bytes = base64.b64decode(value, validate=True)

    if bytes[0:4] != b'%PDF':
        raise ValueError('Missing the PDF file signature')

    filename = get_random_filename('pdf')

    # Write the PDF contents to a local file
    f = open(filename, 'wb')
    f.write(bytes)
    f.close()

    # Lê o conteúdo do arquivo PDF
    text_pdfminer = get_pdfminer_text(filename)
    text_pypdf = get_pypdf_text(filename)

    os.remove(filename)

    return text_pypdf, text_pdfminer

def json_to_excel(json):

    # Converte o Json para DataFrame
    df = pd.DataFrame(json)
    df.astype({'ATUAL': 'float'}).dtypes

    # Gera um nome de arquivo aleatório
    filename = get_random_filename('xlsx')

    # Criar o arquivo Excel
    writer = pd.ExcelWriter(filename, engine='openpyxl')
    df.to_excel(writer, index=False, float_format="%.2f")

    # Ajustar o tamanho das colunas para a largura da maior célula em cada coluna
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    for i, col in enumerate(df.columns):
        column_len = df[col].astype(str).str.len().max()
        column_len = max(column_len, len(col)) + 2
        worksheet.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = column_len

    # Salvar o arquivo Excel
    writer.close()

    with open(filename, "rb") as f:
        content = f.read()

    os.remove(filename)

    return base64.b64encode(content).decode("utf-8")

def update_file_status(response, file_name, success):

    if 'FileStatus' not in response:
        response['FileStatus'] = []

    response['FileStatus'].append({
        'FileName': file_name,
        'Status': "OK" if success else "ERRO"
    })

def extract_info(result, text_pdfminer, array, ativo_idx, valor_idx, vencimento_idx, indexador_idx, data_emissao, nome_arquivo, instituicao, is_coe = False):

    for item in array:

        # Obtém o valor dos atributos
        ativo = '-' if ativo_idx is None else check_ativo_name(text_pdfminer, item[ativo_idx])
        classificacao = "Multimercado" if is_coe else obtem_classificacao(ativo)
        valor_atual = "-" if valor_idx is None else re.sub('\s+',' ', item[valor_idx].strip())
        data_vencimento = "-" if vencimento_idx is None else re.sub('\s+',' ', item[vencimento_idx].strip())
        indexador = "-" if indexador_idx is None else re.sub('^\+\s*','', re.sub('\s+',' ', item[indexador_idx].strip()))

        # Garante que o ativo capturado possua letras
        if ativo != '' and not re.search(r'[a-zA-Z]', ativo):
            continue
        
        # Realiza formatações adicionais
        ativo = "***" if ativo == '' else ativo
        ativo = "COE - " + ativo if is_coe else ativo
        valor_atual = float(valor_atual.replace('.','').replace(',','.'))
        data_vencimento = data_vencimento if re.search(r'\d{2}\/\d{2}\/\d{4}', data_vencimento) else "-"

        result.append({
            'CLASSIFICAÇÃO': classificacao,
            'ATIVO': ativo,
            'EXPOSIÇÃO': "Real",
            'INSTITUIÇÃO': instituicao,
            'ORIGEM EXTRATO': nome_arquivo,
            'ATUAL': valor_atual,
            'DATA': data_emissao,
            'ON/OFF': "ON",
            'DATA DE VENCIMENTO': data_vencimento,
            'INDEXADOR': indexador
        })

In [34]:
def obtem_extrato_consolidado_modal(file_name, text_pypdf, text_pdfminer):

    # Cria o array resultante
    result = []

    # Regex para obter a Data de Emissão
    regex = "(?<=Per.odo de refer.ncia \d{2}\/\d{2}\/\d{4} a )\d{2}\/\d{2}\/\d{4}"
    data_emissao = re.search(regex, text_pdfminer, flags=(re.IGNORECASE)).group(0)

    # Regex para obter todas as rendas fixas
    regex = r'\n[\d\/]*([a-zA-Z].+?)\s*\d{2}\/\d{2}\/\d{4}\s*(\d{2}\/\d{2}\/\d{4})\s*R\$\s*[\d,.]+[.,]\d{2}\s*[\d.]+(\s*)R\$\s*([\d,.]+[.,]\d{2})\s*R\$\s*[\d,.]+[.,]\d{2}\s*[\d.,]+'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, text_pdfminer, tuple_array, 0, 3, 1, None, data_emissao, file_name, "Modal")   

    # Regex para obter todas as ações
    regex = r'(BRUTO|\,\d\d)\s*([a-zA-Z].+?\-(.|\n)*?[A-Z]{4}(3|4|5|6|11|32|33|34|35))\s*[\d.]+\s*R\$\s*[\d.]+\,\d\d\s*R\$\s*([\d.]+\,\d\d)'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, text_pdfminer, tuple_array, 1, 4, None, None, data_emissao, file_name, "Modal")
    
    # Regex para obter todas os CRAs
    regex = r'(L.QUIDO|\,\d\d)\s*([a-zA-Z].+?\-(.|\n)*?)\d{2}\/\d{2}\/\d{4}\s*[\d.]+\s*R\$\s*([\d.]+\,\d\d)\s*R\$\s*[\d.]+\,\d\d\s*R\$\s*[\d.]+\,\d\d'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, text_pdfminer, tuple_array, 1, 3, None, None, data_emissao, file_name, "Modal")

    return result

def obtem_posicao_performance_xp(file_name, text_pypdf, text_pdfminer):

    # Cria o array resultante
    result = []

    # Regex para obter a Data de Emissão
    regex = "(?<=Data da consulta: )\d\d\/\d\d\/\d{4}"
    data_emissao = re.search(regex, text_pypdf, flags=(re.IGNORECASE)).group(0)
    
    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa
    regex = r'(((?!\d{2}\/\d{2}\/\d{4}).)*)\d{2}\/\d{2}\/\d{4}\s*(\d{2}\/\d{2}\/\d{4}|\-)\s*(\d{2}\/\d{2}\/\d{4})([A-Z\-\+\s]*\s*[\d,.]+%[A-Z\-\s]*)([\d.]+\,\d\d)[\d.]+\,\d\d'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, text_pdfminer, tuple_array, 0, 5, 3, 4, data_emissao, file_name, "XP Investimentos")

    # Regex para obter todas as linhas que possuírem ativos de COE
    regex = r'(((?!\d{2}\/\d{2}\/\d{4}).)*)\d{2}\/\d{2}\/\d{4}[\s\d.]+\,\d\d([\d.]+\,\d\d)[\d.]+\,\d\d'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, text_pdfminer, tuple_array, 0, 2, None, None, data_emissao, file_name, "XP Investimentos")

    # Regex para obter todas as linhas que possuírem ativos de COE
    regex = r'(((?!\d{2}\/\d{2}\/\d{4}).)*)\d{2}\/\d{2}\/\d{4}\s*(\d{2}\/\d{2}\/\d{4})\s*[\d.]+\,\d\d[\d.]+\,\d\d([\d.]+\,\d\d)'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))   
    extract_info(result, text_pdfminer, tuple_array, 0, 3, 2, None, data_emissao, file_name, "XP Investimentos", True)

    # Regex para obter todas as linhas que possuírem ativos de Previdência
    regex = r'(((?!\d{2}\/\d{2}\/\d{4}).)*)\d{2}\/\d{2}\/\d{4}\s*[\d.]+\,\d{2}\s*[\d.]+\s*([\d.]+\,\d{2})'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))  
    print(tuple_array)
    extract_info(result, text_pdfminer, tuple_array, 0, 2, None, None, data_emissao, file_name, "XP Investimentos")

    # Regex para obter todas as linhas que possuírem ativos de ??
    regex = r'([A-Z]{4}(3|4|5|6|11|32|33|34|35))[\d\s.]*\,\d\d\s*[\d.]+\,\d\d\s*[\d\-\+]+\,\d\d\s*([\d.]+\,\d\d)'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))   
    extract_info(result, text_pdfminer, tuple_array, 0, 2, None, None, data_emissao, file_name, "XP Investimentos")    

    return result

def obtem_posicao_consolidada_xp(file_name, text_pypdf, text_pdfminer):

    # Cria o dataFrame resultante
    result = []

    # Regex para obter a Data de Emissão
    regex = "(?<=Data da Consulta: )\d{2}\/\d{2}\/\d{4}"
    data_emissao = re.search(regex, text_pypdf, flags=(re.IGNORECASE)).group(0)

    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa
    regex = r'\n[\d\/]*([A-Z].+?\s*-\s*[A-Z]{3}\/\d{4})\s*\d{2}\/\d{2}\/\d{4}\s*\d{2}\/\d{2}\/\d{4}\s*(\d{2}\/\d{2}\/\d{4})\s*([A-Z\-\+\s]*\s*[\d,.]+%[A-Z\-\s]*)\d+\s+\d+\s*R\$\s*[\d,.]+[.,]\d{2}\s*R\$\s*([\d,.]+[.,]\d{2})\s*R\$\s*[\d,.]+[.,]\d{2}'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, text_pdfminer, tuple_array, 0, 3, 1, 2, data_emissao, file_name, "XP Investimentos")   

    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa sem carência
    regex = r'\n[\d\/]*([A-Z].+?\s*-\s*[A-Z]{3}\/\d{4})\s*\d{2}\/\d{2}\/\d{4}\s*\-\s*(\d{2}\/\d{2}\/\d{4})\s*([A-Z\-\+\s]*\s*[\d,.]+%[A-Z\-\s]+)\d+\s+\d+\s*R\$\s*[\d,.]+[.,]\d{2}\s*R\$\s*([\d,.]+[.,]\d{2})\s*R\$\s*[\d,.]+[.,]\d{2}'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, text_pdfminer, tuple_array, 0, 3, 1, 2, data_emissao, file_name, "XP Investimentos")  

    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa Pós-Fixados
    regex = r'\n[\d\/]*([A-Z]((?!\d{2}\/\d{2}\/\d{4}).)+)\s*\d{2}\/\d{2}\/(\d{4})\s*[\d,.]+\s*[\d,.]+\s*R\$\s*[\d,.]+(\s*)R\$\s*([\d,.]+)\s*R\$\s*[\d,.]+'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, text_pdfminer, tuple_array, 0, 4, None, None, data_emissao, file_name, "XP Investimentos")  

    # Regex para obter todas as linhas que possuírem Fundos Imobiliários
    regex = r'([A-Z]{4}(3|4|5|6|11|32|33|34|35))\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+R\$\s*[\d,.]+[.,]\d{2}(\s*)R\$\s*([\d,.]+[.,]\d{2})'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, text_pdfminer, tuple_array, 0, 3, None, None, data_emissao, file_name, "XP Investimentos")

    # Regex para obter todas as linhas que possuírem COEs
    regex = r'\n[\d\/]*([A-Z]((?!\d{2}\/\d{2}\/\d{4}).)+)\s*-\s*[\w\s,.]*\s*-\s*\s*\d{2}\.\d{2}\.\d{4}\s*[\w\s,.]*\d{2}\/\d{2}\/\d{4}\s+(\d{2}\/\d{2}\/\d{4})\s+\d+\s*R\$\s*[\d,.]+[.,]\d{2}\s*R\$\s*[\d,.]+[.,]\d{2}(\s*)R\$\s*([\d,.]+[.,]\d{2})'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, text_pdfminer, tuple_array, 0, 4, 2, None, data_emissao, file_name, "XP Investimentos", True)

    # Regex para obter todas as linhas que possuírem Ações
    regex = r'([A-Z]{4}(3|4|5|6|11|32|33|34|35))\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+R\$\s*[\d,.]+[.,]\d{2}(\s*)R\$\s*([\d,.]+[.,]\d{2})'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, text_pdfminer, tuple_array, 0, 3, None, None, data_emissao, file_name, "XP Investimentos")

    return result

def obtem_extrato_cotista_xp(file_name, text_pypdf, text_pdfminer):

    # Regex para obter a data de emissão
    regex = "(?<=Movimenta..o de \d{2}\/\d{2}\/\d{4} a )\d{2}\/\d{2}\/\d{4}"
    data_emissao = re.search(regex, text_pdfminer, flags=(re.IGNORECASE)).group(0)

    # Regex para obter o texto situado entre o termo "POSIÇÃO CONSOLIDADA" e o termo "Emissão"
    regex = "(?<=POSI..O CONSOLIDADA).*?(?=Emissão:)"
    result_1 = re.search(regex, text_pdfminer, flags=(re.IGNORECASE | re.DOTALL)).group(0)

    # Regex para obter todas as linhas que possuírem pelo menos uma letra e um espaço
    regex = "^.*[a-zA-Z] .*$"
    result_2 = re.finditer(regex, result_1, flags=(re.MULTILINE))

    # Regex para obter o texto situado entre o termo "POSIÇÃOCONSOLIDADA" e o termo "TotalnaInstituição"
    regex = "(?<=POSI..OCONSOLIDADA\n).*?(?=TotalnaInstituição)"
    result_3 = re.search(regex, text_pypdf, flags=(re.IGNORECASE | re.DOTALL)).group(0)

    # Substitui os espaços por um ponto e vírgula, para simular uma tabela CSV
    result_3 = result_3.replace(" ",";")

    # Corrige termos com espaço
    for result in result_2:
        value = result.group(0)
        key = value.replace(" ","")
        result_3 = result_3.replace(key,value)

    # Converte a tabela CSV para um array de valores
    df_array = pd.read_csv(io.StringIO(result_3), sep=";").to_numpy()
    
    # Cria o array resultante
    result = []
    extract_info(result, text_pdfminer, df_array, 0, 4, None, None, data_emissao, file_name, "XP Investimentos")

    return result

def execute_extraction(file_name, text_pypdf, text_pdfminer):

    # Mapeia os métodos de extração
    extraction_map = {
        "Extrato de Cotista XP": obtem_extrato_cotista_xp,
        "Posição Consolidada XP": obtem_posicao_consolidada_xp,
        "Posição e Performance XP": obtem_posicao_performance_xp,
        "Extrato Consolidado Modal": obtem_extrato_consolidado_modal
    }

    # Obtém o tipo do extrato
    tipo_extrato = get_tipo_extrato(text_pypdf, text_pdfminer)
    
    # Invoca o método de extração de interesse
    if tipo_extrato in extraction_map:
        return extraction_map[tipo_extrato](file_name, text_pypdf, text_pdfminer)

    return []

In [38]:
# @Flask(__name__).route('/obtem-ativos', methods=['POST'])
def obtem_ativos():

    # Obtém a requisição 
    request_data = get_response() #json.loads(request.data)

    # Cria as variáveis resultantes
    result, response = [], {"FileStatus": [], "Value": ""}

    # Percorre cada um dos arquivos do array
    for res in request_data:
        
        # Obtém o texto presente no pdf 
        file_name = res['FileName']
        text_pypdf, text_pdfminer = convert_base64_pdf_to_text(res['Value'])

        # Obtém o tamanho do array antes da inserção dos novos ativos
        len_before = len(result)

        # Executa a extração do PDF
        try:
            result = result + execute_extraction(file_name, text_pypdf, text_pdfminer)
        except Exception as e:
            print(e)

        # Atualiza o status de extração do arquivo atual
        update_file_status(response, file_name, (len(result) > len_before))

    # Caso o tamanho do DataFrame seja maior que zero, retorna o base64 do consolidado
    if (len(result) > 0):
        response["Value"] = json_to_excel(result)

    return response

In [39]:
response = obtem_ativos()
response

{'FileStatus': [{'FileName': 'Extrato XP - Cliente 01.pdf', 'Status': 'OK'},
  {'FileName': 'Extrato XP - Cliente 02.pdf', 'Status': 'ERRO'},
  {'FileName': 'Gestao_ExtratoConsolidado Modal - Cliente 04.pdf',
   'Status': 'OK'}],
 'Value': 'UEsDBBQAAAAIAHCOSVYHQU1igQAAALEAAAAQAAAAZG9jUHJvcHMvYXBwLnhtbE2OPQsCMRBE/8pxvbdBwUJiQNBSsLIPexsvkGRDskJ+vjnBj24ebxhG3wpnKuKpDi2GVI/jIpIPABUXirZOXaduHJdopWN5ADvnkc6Mz0hJYKvUHqgJpZnmTf4Ojkafcg4erXhO5uqxcGUnw6UhBQ3/cm3eqdQ17yb1lh/W8DtpXlBLAwQUAAAACABwjklWlLv6CO8AAAArAgAAEQAAAGRvY1Byb3BzL2NvcmUueG1szZLPTsMwDIdfBeXeOm3REFGXC9NOICExCcQtSrwtovmjxKjd29OWrROCB+AY+5fPnyW3OgodEj6nEDGRxXwzuM5noeOaHYmiAMj6iE7lckz4sbkPySkan+kAUekPdUCoOV+BQ1JGkYIJWMSFyGRrtNAJFYV0xhu94ONn6maY0YAdOvSUoSorYHKaGE9D18IVMMEIk8vfBTQLca7+iZ07wM7JIdsl1fd92Tdzbtyhgrenx5d53cL6TMprHH9lK+gUcc0uk1+bh81uy2TN66bgdcHvd9WdaLi4Xb1Prj/8rsIuGLu3/9j4Iihb+HUX8gtQSwMEFAAAAAgAcI5JVplcnCMQBgAAnCcAABMAAAB4bC90aGVtZS90aGVtZTEueG1s7Vpbc9o4FH7vr9B4Z/ZtC8Y2gba0E3Npdtu0mYTtTh+FEViNbHlkkYR/v0c2EMuWDe2STbqbPAQs6fv

In [37]:
if response['Value'] != '':

    f = open("Ativos.xlsx", 'wb')
    f.write(base64.b64decode(response['Value'], validate=True))
    f.close()
    
    files.download("Ativos.xlsx")